In [2]:
import ccdproc
from pathlib import Path
from astropy.nddata import CCDData
from lbcred import image, reduce

In [7]:
#### Call function to get images
lbc_raw = Path('/Volumes/WD-EXFAT/Images/lbc-raw/')
lbc_red = Path('/Volumes/WD-EXFAT/Images/','lbc-reduced')
lbc_red.mkdir(exist_ok=True)
all_files = ccdproc.ImageFileCollection(lbc_raw,ext=1)
all_files.summary['file','imagetyp','filter','object','exptime','naxis1','naxis2']

file,imagetyp,filter,object,exptime,naxis1,naxis2
str25,str6,str8,str15,float64,int64,int64
lbcb.20191220.003943.fits,flat,SDT_Uspe,SkyFlat,1.284,2304,4608
lbcb.20191220.004029.fits,flat,SDT_Uspe,SkyFlat,1.266,2304,4608
lbcb.20191220.004110.fits,flat,SDT_Uspe,SkyFlat,1.284,2304,4608
lbcb.20191220.004155.fits,flat,SDT_Uspe,SkyFlat,1.266,2304,4608
lbcb.20191220.004237.fits,flat,SDT_Uspe,SkyFlat,1.284,2304,4608
lbcb.20191220.004615.fits,flat,B-BESSEL,SkyFlat,1.284,2304,4608
lbcb.20191220.004658.fits,flat,B-BESSEL,SkyFlat,1.266,2304,4608
lbcb.20191220.004744.fits,flat,B-BESSEL,SkyFlat,1.284,2304,4608
lbcb.20191220.004910.fits,flat,B-BESSEL,SkyFlat,1.284,2304,4608


In [10]:
#### Call function to calibrate bias images and make master bias
biases_raw = all_files.files_filtered(include_path=True,imagetyp='zero')
# Loop through bias images and calibrate each one (subtract overscan and trim)
# Combine all bias images
# Return 'master' bias

In [4]:
# Call function to do biasing

In [5]:
# Call function to do flat fielding

In [6]:
# Call function to stack images/do astrometry

In [27]:
keys = ['imagetyp','object','filter',]
collection = ccdproc.ImageFileCollection(lbc_raw, keywords=keys, ext=1)
sub_collection = collection.files_filtered(imagetyp='object',filter='B-BESSEL',object='*')
collection.values('object',unique=True)

['NGC628KDG10',
 'KK15',
 'KK17',
 'DDO13_UGC01171',
 'M81blob',
 'TYC 627-39-1',
 'faint',
 'UGC685',
 'BinoBias',
 'TYC 1205-1173-1',
 'DDO043',
 'TYC 1205-1342-1',
 'NGC628 smiley',
 'AGC748778',
 'NGC628LSB',
 'SkyFlat']

In [31]:
print(collection.summary)

           file           imagetyp  object  filter 
------------------------- -------- ------- --------
lbcb.20191220.003943.fits     flat SkyFlat SDT_Uspe
lbcb.20191220.004029.fits     flat SkyFlat SDT_Uspe
lbcb.20191220.004110.fits     flat SkyFlat SDT_Uspe
lbcb.20191220.004155.fits     flat SkyFlat SDT_Uspe
lbcb.20191220.004237.fits     flat SkyFlat SDT_Uspe
lbcb.20191220.004615.fits     flat SkyFlat B-BESSEL
lbcb.20191220.004658.fits     flat SkyFlat B-BESSEL
lbcb.20191220.004744.fits     flat SkyFlat B-BESSEL
lbcb.20191220.004910.fits     flat SkyFlat B-BESSEL
lbcb.20191220.005038.fits     flat SkyFlat V-BESSEL
                      ...      ...     ...      ...
lbcr.20191222.052427.fits   object    KK17 R-BESSEL
lbcr.20191222.052542.fits   object    KK17 R-BESSEL
lbcr.20191222.053122.fits   object    KK17 R-BESSEL
lbcr.20191222.053239.fits   object    KK17 R-BESSEL
lbcr.20191222.053819.fits   object    KK17 R-BESSEL
lbcr.20191222.053944.fits   object    KK17 R-BESSEL
lbcr.2019122

In [48]:
imtype='*'
objectname='UGC685'
filt='*'

images = collection.filter(imagetyp=imtype, object=objectname, filter=filt)
options = {}
options['imtype'] = images.values('imagetyp', unique=True)
options['objectname'] = images.values('object', unique=True)
options['filt'] = images.values('filter', unique=True)
print('Options','-------',sep='\n')
print('\n','imtype: ', options['imtype'],sep='')
print('\n','objectname: ', options['objectname'], sep='')
print('\n', 'filt: ', options['filt'], sep='')

Options
-------

imtype: ['object']

objectname: ['UGC685']

filt: ['B-BESSEL', 'R-BESSEL']
